# Hyperparameter tuning with AI Platform 

In [1]:
import os
import time

In [2]:
TRAINING_DATA_PATH = 'gs://jk-demo-datasets/gasdata/training.csv'
TESTING_DATA_PATH = 'gs://jk-demo-datasets/gasdata/testing.csv'
REGION = "us-central1"
ARTIFACT_BUCKET = 'gs://jk-demo-artifacts'
JOBDIR_BUCKET = 'gs://jk-demo-jobdir'

## Create a training application package
### Create a training module

In [3]:
TRAINING_APP_FOLDER = '../hypertune_app/trainer'

os.makedirs(TRAINING_APP_FOLDER, exist_ok=True)
!touch $TRAINING_APP_FOLDER/__init__.py

In [4]:
%%writefile $TRAINING_APP_FOLDER/train.py

import logging
import os
import subprocess
import sys

import fire
import numpy as np
import pandas as pd

import hypertune

from sklearn.model_selection import cross_val_score
from sklearn.externals import joblib
from sklearn.decomposition import PCA
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler


def train(job_dir, data_path, n_components, alpha):
    
    
  # Load data from GCS
  df_train = pd.read_csv(data_path)

  y = df_train.octane
  X = df_train.drop('octane', axis=1)
    
  # Configure a training pipeline
  pipeline = Pipeline([
    ('scale', StandardScaler()),
    ('reduce_dim', PCA(n_components=n_components)),
    ('regress', Ridge(alpha=alpha))
  ])

  # Calculate the performance metric
  scores = cross_val_score(pipeline, X, y, cv=10, scoring='neg_mean_squared_error')
    
  # Log it with hypertune
  hpt = hypertune.HyperTune()
  hpt.report_hyperparameter_tuning_metric(
    hyperparameter_metric_tag='neg_mean_squared_error',
    metric_value=scores.mean()
    )

  # Fit the model on a full dataset
  pipeline.fit(X, y)

  # Save the model
  model_filename = 'model.joblib'
  joblib.dump(value=pipeline, filename=model_filename)
  gcs_model_path = "{}/{}".format(job_dir, model_filename)
  subprocess.check_call(['gsutil', 'cp', model_filename, gcs_model_path], stderr=sys.stdout)
  logging.info("Saved model in: {}".format(gcs_model_path)) 
    
if __name__ == "__main__":
  logging.basicConfig(level=logging.INFO)
  fire.Fire(train)

Overwriting ../hypertune_app/trainer/train.py


### Create hyperparameter configuration file

In [5]:
%%writefile $TRAINING_APP_FOLDER/hptuning_config.yaml 

trainingInput:
  hyperparameters:
    goal: MAXIMIZE
    maxTrials: 50
    maxParallelTrials: 3
    hyperparameterMetricTag: neg_mean_squared_error
    enableTrialEarlyStopping: TRUE 
    params:
    - parameterName: n_components
      type: DISCRETE
      discreteValues: [
          2,
          3,
          4,
          5,
          6,
          7,
          8
          ]
    - parameterName: alpha
      type: DOUBLE
      minValue:  0.0001
      maxValue:  0.1
      scaleType: UNIT_LINEAR_SCALE
      

Overwriting ../hypertune_app/trainer/hptuning_config.yaml


### Configure dependencies

In [6]:
%%writefile $TRAINING_APP_FOLDER/../setup.py

from setuptools import find_packages
from setuptools import setup

REQUIRED_PACKAGES = ['fire', 'gcsfs', 'cloudml-hypertune']

setup(
    name='trainer',
    version='0.1',
    install_requires=REQUIRED_PACKAGES,
    packages=find_packages(),
    include_package_data=True,
    description='My training application package.'
)

Overwriting ../hypertune_app/trainer/../setup.py


## Submit a training job

In [7]:
JOB_NAME = "JOB_{}".format(time.strftime("%Y%m%d_%H%M%S"))
SCALE_TIER = "BASIC"
MODULE_NAME = "trainer.train"
RUNTIME_VERSION = "1.14"
PYTHON_VERSION = "3.5"

In [8]:
!gcloud ai-platform jobs submit training $JOB_NAME \
--region $REGION \
--job-dir $JOBDIR_BUCKET/$JOB_NAME \
--package-path $TRAINING_APP_FOLDER \
--module-name $MODULE_NAME \
--scale-tier $SCALE_TIER \
--python-version $PYTHON_VERSION \
--runtime-version $RUNTIME_VERSION \
--config $TRAINING_APP_FOLDER/hptuning_config.yaml \
-- \
--data_path $TRAINING_DATA_PATH 

Job [JOB_20191011_011808] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe JOB_20191011_011808

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs JOB_20191011_011808
jobId: JOB_20191011_011808
state: QUEUED


In [9]:
!gcloud ai-platform jobs describe $JOB_NAME

createTime: '2019-10-11T01:18:14Z'
etag: 0LXRIwT4yek=
jobId: JOB_20191011_011808
startTime: '2019-10-11T01:18:19Z'
state: RUNNING
trainingInput:
  args:
  - --data_path
  - gs://jk-demo-datasets/gasdata/training.csv
  hyperparameters:
    enableTrialEarlyStopping: true
    goal: MAXIMIZE
    hyperparameterMetricTag: neg_mean_squared_error
    maxParallelTrials: 3
    maxTrials: 50
    params:
    - discreteValues:
      - 2.0
      - 3.0
      - 4.0
      - 5.0
      - 6.0
      - 7.0
      - 8.0
      parameterName: n_components
      type: DISCRETE
    - maxValue: 0.1
      minValue: 0.0001
      parameterName: alpha
      scaleType: UNIT_LINEAR_SCALE
      type: DOUBLE
  jobDir: gs://jk-demo-jobdir/JOB_20191011_011808
  packageUris:
  - gs://jk-demo-jobdir/JOB_20191011_011808/packages/440c61ae9600720f4dc86c0470244cbc8f5603a6cc2c10067421e734e5565064/trainer-0.1.tar.gz
  pythonModule: trainer.train
  pythonVersion: '3.5'
  region: us-central1
  runtimeVersion: '1.14'
trainingOutput:
 